In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-09 17:00:00| 225|             82.46|             5|
|2020-01-02 21:00:00|  74| 506.4200000000001|            38|
|2020-01-27 13:00:00|  24|            203.57|             7|
|2020-01-06 16:00:00|  43|343.68000000000006|            21|
|2020-01-05 08:00:00|  55|132.51999999999998|             4|
|2020-01-20 15:00:00|  82|            380.22|            27|
|2020-01-28 22:00:00| 255|217.10999999999999|            13|
|2020-01-27 09:00:00| 177|             102.8|             3|
|2020-01-26 20:00:00|  35|             48.61|             2|
|2020-01-29 08:00:00| 223|            119.93|             8|
|2020-01-16 10:00:00|  82|273.62000000000006|            18|
|2020-01-24 18:00:00| 210|            124.58|             6|
|2020-01-02 23:00:00|  66|            258.49|            11|
|2020-01-14 14:00:00| 22

In [9]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [10]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [11]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [12]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [13]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [14]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [15]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [16]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  68|              null|                null| 7825.069999999994|                  396|
|2020-01-01 00:00:00|  88|              null|                null| 823.8000000000001|                   36|
|2020-01-01 00:00:00|  93|              null|                null|            210.28|                    3|
|2020-01-01 00:00:00| 106|             10.56|                   1|              null|                 null|
|2020-01-01 00:00:00| 248|              null|                null|              25.5|                    1|
|2020-01-01 01:00:00|  13|  

In [17]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [18]:
df_join = spark.read.parquet('data/report/revenue/total')

In [19]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [20]:
df_zones = spark.read.parquet('zones/')

In [21]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [23]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  34|              null|                null|              19.3|                    1|        34| Brooklyn|  Brooklyn Navy Yard|   Boro Zone|
|2020-01-01 00:00:00|  36|            295.34|                  11|            109.17|                    3|        36| Brooklyn|      Bushwick North|   Boro Zone|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|        61| Brooklyn| Crown Heights North|   Boro Zone|
|2020-01-01 00:00:00| 

In [24]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')